In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import sklearn
from sklearn.model_selection import RandomizedSearchCV, ShuffleSplit
import seaborn as sns
import sys
import torch

import warnings
#warnings.filterwarnings('ignore')
#warnings.simplefilter('ignore')

mpl.style.use('seaborn')

In [ ]:
from ZORO import benchmarkfunctions
from ZORO import optimizers

## Generate Data

In [ ]:
# Dimensionality of data -- we tweak this to run our experiments
dim = 1000             #10, 100, 1000, 10000
informative = 1000     # 1,  10,  100,  1000

In [ ]:
from sklearn.datasets import make_classification, make_blobs
# Cluster of points normally distributed (std=1) about vertices
# of an n_informative-dim hypercube with sides of length 2*class_sep

# Features are ordered: n_informative, n_redundant, n_repeated, 
# then random noise
def generate_samples(n_features, n_informative, n_samples=100):
    return make_blobs(
        n_samples=n_samples,
        n_features=n_features,
        centers=2,
        
    )
    """
    return make_classification(
        n_samples=n_samples, 
        n_features=n_features, 
        n_informative=n_informative,
        n_redundant=0, 
        n_repeated=0, 
        n_classes=2, 
        n_clusters_per_class=1, 
        flip_y=0.01, 
        class_sep=2, 
        random_state=42,
        shuffle=False
    )
    """

X, y = generate_samples(dim, informative)

### Plot of Projection onto 2 Dimensions

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=2)
X_reduced = pca.fit_transform(X)

In [ ]:
sns.scatterplot(x=X_reduced[:,0], y=X_reduced[:,1], hue=y)

## Linear Separator

In [ ]:
from sklearn.svm import LinearSVC

clf = LinearSVC(random_state=42, max_iter=1000)
clf.fit(X, y)
print("Number of iterations", clf.n_iter_)
print("Accuracy:", clf.score(X, y))

## ZORO Attack

In [ ]:
# Adversarial Attack Loss
class AttackLoss(object):
    '''An implementation of the sparse quadric function.'''
    def __init__(self, predictor, lamb, norm, x_original, y_true, y_attack):
        self.predictor = predictor
        self.lamb = lamb
        self.norm = norm
        
        self.x_original = x_original
        self.y_true = y_true
        self.y_attack = y_attack
        
    def __call__(self, x_attack):
        ## (f(x + delta) - y_attack + y_true)^2 + lambda ||delta||_0
        prediction = self.predictor.decision_function(x_attack)
                
        return (prediction - self.y_attack + self.y_true)**2 \
              + self.lamb * np.linalg.norm((x_attack - self.x_original), self.norm, axis=1)

In [ ]:
class AdaZOROExperiment:
    
    def __init__(self, step_size=None, delta=None, max_cosamp_iter=None, 
                 cosamp_tol=None, prop_sparsity=None, lamb=None, norm=None,
                 function_budget=None, num_samples_constant=None, phi_cosamp=None,
                 phi_lstsq=None, compessible_constant=None):
        self.step_size = step_size
        self.delta = delta
        self.max_cosamp_iter = max_cosamp_iter
        self.cosamp_tol = cosamp_tol
        self.prop_sparsity = prop_sparsity
        self.lamb = lamb
        self.norm = norm
        self.function_budget = function_budget
        self.num_samples_constant=num_samples_constant
        self.phi_cosamp=phi_cosamp
        self.phi_lstsq=phi_lstsq
        self.compessible_constant=compessible_constant
        
    def score(self, X, y):
        return self.loss
    
    def get_params(self, deep=True):
        return {
            # Parameters for ZORO. 
            "step_size": self.step_size,
            "delta": self.delta,
            "max_cosamp_iter": self.max_cosamp_iter,
            "cosamp_tol": self.cosamp_tol,
            "prop_sparsity": self.prop_sparsity,
            "lamb" : self.lamb,
            "norm" : self.norm,
            "function_budget" : self.function_budget,
            "num_samples_constant": self.num_samples_constant,
            "phi_cosamp": self.phi_cosamp,
            "phi_lstsq": self.phi_lstsq,
            "compessible_constant": self.compessible_constant,
        }
    
    def set_params(self, **kwargs):
        for parameter, value in kwargs.items():
            setattr(self, parameter, value)
        return self
    
    def fit(self, X, y):
        self.report = []
        
        params = {
            "step_size": self.step_size,
            "delta": self.delta,
            "max_cosamp_iter": self.max_cosamp_iter,
            "cosamp_tol": self.cosamp_tol,
            "prop_sparsity": self.prop_sparsity,
            "lamb" : self.lamb,
            "norm" : self.norm,
            "function_budget" : self.function_budget,
            "num_samples_constant": self.num_samples_constant,
            "phi_cosamp": self.phi_cosamp,
            "phi_lstsq": self.phi_lstsq,
            "compessible_constant": self.compessible_constant,
        }
        
        params["sparsity"] = int(params["prop_sparsity"] * X.shape[1])
        params["num_samples"] = int(np.ceil(np.log(X.shape[1])*params["sparsity"]))

        # Compute attack loss for each data point individually
        for i in range(len(X)):
            x0           = X[i, :]
            xx0          = x0.copy()

            label        = y[i]
            label_attack = 1 - y[i]

            obj_func = AttackLoss(
                predictor=clf, 
                lamb=self.lamb, 
                norm=self.norm, 
                x_original=xx0,
                y_true=label,
                y_attack=label_attack
            )

            # initialize optimizer object
            self.report.append([{"evals": 0, "x": x0, "y": label, "loss": obj_func(np.expand_dims(x0, 0))[0]}])
            opt = optimizers.AdaZORO(x0, obj_func, params, function_budget=self.function_budget, function_target=0.001)

            # the optimization routine
            termination = False
            while termination is False:
                # optimization step
                # solution_ZORO = False until a termination criterion is met, in which 
                # case solution_ZORO = the solution found.
                # termination = False until a termination criterion is met.
                # If ZORO terminates because function evaluation budget is met, 
                # termination = B
                # If ZORO terminated because the target accuracy is met,
                # termination= T.

                opt.report("Step")
                evals_ZORO, solution_ZORO, termination = opt.step()

                # save some useful values
                self.report[-1].append({"evals" : evals_ZORO, "x": solution_ZORO, "loss": np.mean(opt.fd)})
                # print some useful values
                opt.report( f'Estimated f(x_{i}): %f  function evals: %d\n' %
                    (np.mean(opt.fd), evals_ZORO) )
        self.loss = sum([self.report[i][-1]["loss"] for i in range(len(self.report))]) / len(self.report)
    

class ZOROExperiment:       
    
    def __init__(self, step_size=None, delta=None, max_cosamp_iter=None, 
                 cosamp_tol=None, prop_sparsity=None, lamb=None, norm=None,
                 function_budget=None):
        self.step_size = step_size
        self.delta = delta
        self.max_cosamp_iter = max_cosamp_iter
        self.cosamp_tol = cosamp_tol
        self.prop_sparsity = prop_sparsity
        self.lamb = lamb
        self.norm = norm
        self.function_budget = function_budget
        
    def score(self, X, y):
        return self.loss
    
    def get_params(self, deep=True):
        return {
            # Parameters for ZORO. 
            "step_size": self.step_size,
            "delta": self.delta,
            "max_cosamp_iter": self.max_cosamp_iter,
            "cosamp_tol": self.cosamp_tol,
            "prop_sparsity": self.prop_sparsity,
            "lamb" : self.lamb,
            "norm" : self.norm,
            "function_budget" : self.function_budget
        }
    
    def set_params(self, **kwargs):
        for parameter, value in kwargs.items():
            setattr(self, parameter, value)
        return self
    
    def fit(self, X, y):
        self.report = []
        
        params = {
            "step_size": self.step_size,
            "delta": self.delta,
            "max_cosamp_iter": self.max_cosamp_iter,
            "cosamp_tol": self.cosamp_tol,
            "prop_sparsity": self.prop_sparsity,
            "lamb" : self.lamb,
            "norm" : self.norm,
            "function_budget" : self.function_budget
        }
        
        params["sparsity"] = int(params["prop_sparsity"] * X.shape[1])
        params["num_samples"] = int(np.ceil(np.log(X.shape[1])*params["sparsity"]))

        # Compute attack loss for each data point individually
        for i in range(len(X)):
            x0           = X[i, :]
            xx0          = x0.copy()

            label        = y[i]
            label_attack = 1 - y[i]

            obj_func = AttackLoss(
                predictor=clf, 
                lamb=self.lamb, 
                norm=self.norm, 
                x_original=xx0,
                y_true=label,
                y_attack=label_attack
            )

            # initialize optimizer object
            self.report.append([{"evals": 0, "x": x0, "y": label, "loss": obj_func(np.expand_dims(x0, 0))[0]}])
            opt = optimizers.ZORO(x0, obj_func, params, function_budget=self.function_budget, function_target=0.001)

            # the optimization routine
            termination = False
            while termination is False:
                # optimization step
                # solution_ZORO = False until a termination criterion is met, in which 
                # case solution_ZORO = the solution found.
                # termination = False until a termination criterion is met.
                # If ZORO terminates because function evaluation budget is met, 
                # termination = B
                # If ZORO terminated because the target accuracy is met,
                # termination= T.

                evals_ZORO, solution_ZORO, termination = opt.step()

                # save some useful values
                self.report[-1].append({"evals" : evals_ZORO, "x": solution_ZORO, "loss": np.mean(opt.fd)})
                # print some useful values
                #opt.report( f'Estimated f(x_{i}): %f  function evals: %d\n' %
                #    (np.mean(opt.fd), evals_ZORO) )
        self.loss = sum([self.report[i][-1]["loss"] for i in range(len(self.report))]) / len(self.report)

### Hyperparameter Search

In [ ]:
# Parameters to search for ZORO attack
params = {
    "step_size": [1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2],
    "delta": [1e-3, 1e-4, 1e-5], 
    "max_cosamp_iter": [5, 10, 15, 20],
    "cosamp_tol": [0.5], 
    "prop_sparsity": [0.05, 0.10, 0.15, 0.20, 0.25], 
    "lamb" : [0.1], 
    "norm" : [2],
    "function_budget": [5e4] # for hyperparameter tuning, we give this as a budget
}

adaparams = {
    "step_size": [1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2],
    "delta": [1e-3, 1e-4, 1e-5], 
    "max_cosamp_iter": [5, 10, 15, 20],
    "cosamp_tol": [0.5], 
    "prop_sparsity": [0.05, 0.10, 0.15, 0.20, 0.25], 
    "lamb" : [0.1], 
    "norm" : [2],
    "function_budget": [5e3], # for hyperparameter tuning, we give this as a budget
    "num_samples_constant": [dim], 
    "phi_cosamp": [0.2, 0.4, 0.6, 0.8],
    "phi_lstsq": [0.05, 0.1, 0.15, 0.20, 0.25],
    "compessible_constant": [1, 1.1, 1.25, 1.5, 2]
}

In [ ]:
clf_search = sklearn.model_selection.RandomizedSearchCV(
    estimator = AdaZOROExperiment(),
    param_distributions = adaparams,
    n_iter = 100, # Run 100 random trials
    n_jobs = 20, # Run 20 jobs at once
    refit = False,
    cv = ShuffleSplit(n_splits=1, train_size=16, random_state=42), # We attack the same 16 examples for every trial
    error_score=np.nan
)

In [ ]:
# With dimension = 1000, this takes about 30 minutes to run on a machine with 32 CPU cores and 64 GB of memory.
search_results = clf_search.fit(X, y) # Make sure to clear the output of this cell before saving

In [ ]:
import pandas as pd
# We see NaNs when numerical errors due to overflow occur (indicates a terrible hyperparam combination)
pd.DataFrame(search_results.cv_results_).sort_values("mean_test_score")

#### Train Best Model With Higher Budget

In [ ]:
rs = ShuffleSplit(n_splits=1, train_size=16, random_state=42)
# Recover the exact indices used for training (kind of hacky)
for train_index, test_index in rs.split(X):
    X_sel, y_sel = X[train_index], y[train_index]

In [ ]:
best_params = search_results.cv_results_["params"][0]
best_params.update({"function_budget" : 5e5})
best_exp = ZOROExperiment(**best_params)
best_exp.fit(X_sel[:16,:], y_sel[:16])

#### Save Reports

In [ ]:
torch.save(best_exp.report, "gaussian_d1000_20220616_report.pt")

### Reload Best Attacker Data and Plot Graphs

#### Fig 1

PCA of Data Points, Four Groupings:
1. Original Cluster 1
2. Original Cluster 2
3. Attack Points 1->2
4. Attack Points 2->1

#### Fig 2

PCA of Example Point:
1. Original Cluster 1
2. Original Cluster 2
3. Trace of Attack over Iterations

#### Fig 3

Distance from Hyperplane (Vector Norm)

#### Fig 4

Distance from Input to Attack Vector (Vector Norm)

#### Fig 5

Movement in Hyperplane Direction Norm versus Total Movement Norm